In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense, Conv2D, MaxPooling2D, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import splitfolders

input_dir = './DATA/simpsons_dataset'
output_dir = './DATA/data/'

splitfolders.ratio(input_dir, output=output_dir, seed=1337, ratio=(.9, .1))

In [ ]:
train_dir = './DATA/data/train/'
val_dir = './DATA/data/val/'

In [ ]:
image_size = (64, 64, 3)
epochs = 50
batch_size = 128

labels = os.listdir(train_dir)

In [ ]:
# 이미지 증식
# shear_range : 반시계 방향으로 회전
# horizontal_flip : 좌우 방향으로 뒤집기
# rotation_range : 이미지 회전값
# width_shift_range : 좌우 이동
# height_shift_range : 상하 이동

train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# 폴더에서 이미지를 가져와 제너레이터 객체로 정의
training_set = train_datagen.flow_from_directory(train_dir,
                                                 target_size = image_size[:2],
                                                 batch_size=batch_size,
                                                 class_mode = 'sparse',
                                                 color_mode = 'rgb')

val_set = test_datagen.flow_from_directory(val_dir,
                                           target_size = image_size[:2],
                                           batch_size=batch_size,
                                           class_mode = 'sparse',
                                           color_mode = 'rgb')

In [ ]:
model = Sequential()
model.add(Input(shape=image_size))
model.add(Conv2D(32, (3, 3), padding = 'same', activation = 'relu'))
model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(256, (3, 3), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(1024, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(labels), activation = 'softmax'))

In [ ]:
input_layer = Input(shape=image_size)

x1 = Conv2D(32, (3, 3), padding = 'same', activation = 'relu')(input_layer)
x2 = Conv2D(32, (3, 3), activation = 'relu')(x1)
x3 = MaxPooling2D(pool_size=(2, 2))(x2)
x4 = Dropout(0.2)(x3)
x5 = Conv2D(64, (3, 3), padding = 'same', activation = 'relu')(x4)
x6 = MaxPooling2D(pool_size=(2, 2))(x5)
x7 = Dropout(0.2)(x6)
x8 = Conv2D(256, (3, 3), padding = 'same', activation = 'relu')(x7)
x9 = MaxPooling2D(pool_size=(2, 2))(x8)
x10 = Dropout(0.2)(x9)
x11 = Flatten()(x10)
x12 = Dense(1024, activation = 'relu')(x11)
x13 = Dropout(0.5)(x12)
output_layer = Dense(len(labels), activation = 'softmax')(x13)

model = Model(inputs=input_layer, outputs = output_layer)

In [ ]:
stopCB = EarlyStopping(monitor='val_acc', patience=5)

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])

In [ ]:
model.summary()

In [ ]:
stopCB = EarlyStopping(patience=5, monitor='val_acc')

checkpoint_filepath = './model.h5'
ModelCP = ModelCheckpoint(filepath=checkpoint_filepath,
                          save_weights_only = False,
                          monitor = 'val_acc',
                          mode = 'max',
                          save_best_only = True)

In [ ]:
history = model.fit(training_set,
                    epochs=epochs,
                    validation_data = val_set,
                    callbacks = [stopCB, ModelCP])

In [ ]:
acc_loss_df = pd.DataFrame(history.history)
acc_loss_df

In [ ]:
plt.figure(figsize=(15, 7))
plt.subplot(1,2,1)
plt.plot(acc_loss_df.iloc[:, [0, 2]])
plt.title('LOSS')
plt.xlabel('EPOCHS')
plt.legend(['train', 'valid'], fontsize=15)

plt.subplot(1,2,2)
plt.plot(acc_loss_df.iloc[:, [1, 3]])
plt.title('ACCURACY')
plt.xlabel('EPOCHS')
plt.legend(['train', 'valid'], fontsize=15)
plt.show()

In [ ]:
# loading best model
saved_model = load_model('./model.h5')

In [ ]:
img_path = './DATA/kaggle_simpson_testset/kaggle_simpson_testset/krusty_the_clown_10.jpg'
img = load_img(img_path, target_size=(64, 64, 3))
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
def predict(img):
    img_array = img_to_array(img)
    img_batch = np.expand_dims(img_array, axis=0)
    pred = saved_model.predict(img_batch)
    print(pred)
    pred = labels[np.argmax(pred)]
    return pred

In [ ]:
predict(img)